In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("car_evaluation.csv")
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [4]:
df.shape

(1727, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   vhigh    1727 non-null   object
 1   vhigh.1  1727 non-null   object
 2   2        1727 non-null   object
 3   2.1      1727 non-null   object
 4   small    1727 non-null   object
 5   low      1727 non-null   object
 6   unacc    1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [6]:
col_names=['buying_cost','maintain_cost','doors','persons','lug_boost','safety','class']
df.columns=col_names
col_names

['buying_cost',
 'maintain_cost',
 'doors',
 'persons',
 'lug_boost',
 'safety',
 'class']

In [7]:
df

,buying_cost,maintain_cost,doors,persons,lug_boost,safety,class
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   buying_cost    1727 non-null   object
 1   maintain_cost  1727 non-null   object
 2   doors          1727 non-null   object
 3   persons        1727 non-null   object
 4   lug_boost      1727 non-null   object
 5   safety         1727 non-null   object
 6   class          1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [9]:
col_names=['buying_cost','maintain_cost','doors','persons','lug_boost','safety','class']
for col in col_names:
    print(df[col].value_counts())

buying_cost
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64
maintain_cost
high     432
med      432
low      432
vhigh    431
Name: count, dtype: int64
doors
3        432
4        432
5more    432
2        431
Name: count, dtype: int64
persons
4       576
more    576
2       575
Name: count, dtype: int64
lug_boost
med      576
big      576
small    575
Name: count, dtype: int64
safety
med     576
high    576
low     575
Name: count, dtype: int64
class
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64


In [10]:
df.isnull().sum()

buying_cost      0
maintain_cost    0
doors            0
persons          0
lug_boost        0
safety           0
class            0
dtype: int64

In [11]:
df['class'].unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [12]:
df['class'].value_counts()

class
unacc    1209
acc       384
good       69
vgood      65
Name: count, dtype: int64

In [13]:
df['doors'].replace("5more","6",inplace=True)

In [14]:
df["persons"].replace("more","7",inplace=True)

In [15]:
x=pd.get_dummies(df.drop('class',axis=1),drop_first=True)
y=df["class"]

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=6)

In [17]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=6)
model.fit(x_train,y_train) 

RandomForestClassifier(random_state=6)

In [18]:
ypred_train=model.predict(x_train)
ypred_test=model.predict(x_test)

In [19]:
from sklearn.metrics import accuracy_score
print("Train accuricy:",accuracy_score(ypred_train,y_train))
print("Test accuracy:",accuracy_score(ypred_test,y_test))

Train accuricy: 1.0
Test accuracy: 0.8988439306358381


In [20]:
from sklearn.model_selection import cross_val_score
print("Cross validation score:",cross_val_score(model,x,y,cv=5).mean())

Cross validation score: 0.7307598224009383


In [21]:
from sklearn.model_selection import  GridSearchCV

estimator= RandomForestClassifier(random_state=0)

param_grid={'n_estimators':list(range(1,101))}

grid= GridSearchCV(estimator,param_grid,scoring="accuracy",cv=5)

grid.fit(x_train,y_train)

grid.best_params_

{'n_estimators': 69}

In [22]:
grid.best_estimator_.feature_importances_

array([0.06036359, 0.05111235, 0.0348825 , 0.05478616, 0.04632488,
       0.0456347 , 0.02846206, 0.03221794, 0.03031794, 0.11492872,
       0.11871551, 0.03569761, 0.06940441, 0.20229376, 0.07485787])

In [23]:
# Extract feature importances from the best estimator in grid
feats_imp = pd.DataFrame(data=grid.best_estimator_.feature_importances_, index=x.columns, columns=["Feature Importance"])

# Filter out features with non-zero importance
important_feats = feats_imp[feats_imp['Feature Importance'] > 0]

# Get a list of important feature names
important_features_list = important_feats.index.tolist()


In [24]:
x_imp =x[important_features_list]

x_train,x_test,y_train,y_test=train_test_split(x_imp,y,test_size=0.2,random_state=6)

## Model with best hyperparametrers
final_rf_model=RandomForestClassifier(n_estimators=12,random_state=0)
final_rf_model.fit(x_train,y_train)

ypred_train=final_rf_model.predict(x_train)
ypred_test=final_rf_model.predict(x_test)

## Evaluation

print("Train accuracy:",accuracy_score(ypred_train,y_train))
print("Test accuracy:",accuracy_score(ypred_test,y_test))
print("Cross validation score:",cross_val_score(final_rf_model,x_imp,y,cv=5).mean())

Train accuracy: 0.997103548153512
Test accuracy: 0.8757225433526011
Cross validation score: 0.6322208260031834


## Encoding

In [28]:
df["buying_cost"]=df["buying_cost"].map({"low":0,'med':1,'high':2,'vhigh':3})
df["maintain_cost"]=df["maintain_cost"].map({'low':0,'med':1,'high':2,'vhigh':3})
df['lug_boost']=df['lug_boost'].map({'small':0,'mid':1,'big':2})
df['safety']=df['safety'].map({'low':0,'med':1,'high':2})
df['class']=df['class'].map({'unacc':0,'acc':1,'good':2,'vgood':3})

## Find the best random state model

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize lists to store metrics for different random states
Train = []
Test = []
CV = []

# Loop over random states from 1 to 100
for i in range(1, 101):
    # Split the data with the current random state
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
    
    # Initialize and train the logistic regression model
    log_default = LogisticRegression()
    log_default.fit(x_train, y_train)
    
    # Make predictions
    ypred_train = log_default.predict(x_train)
    ypred_test = log_default.predict(x_test)
    
    # Calculate accuracy scores
    Train.append(accuracy_score(y_train, ypred_train))
    Test.append(accuracy_score(y_test, ypred_test))
    
    # Calculate cross-validation scores
    cv_scores = cross_val_score(log_default, x_train, y_train, cv=5, scoring='accuracy')
    CV.append(cv_scores.mean())

# Create a DataFrame to store all the metrics
em = pd.DataFrame({"Train": Train, "Test": Test, "CV": CV})

# Filter the DataFrame based on the specified criteria
gm = em[(abs(em['Train'] - em['Test']) <= 0.05) & (abs(em['Test'] - em['CV']) <= 0.05)]

# Find the index of the best random state
if not gm.empty:
    rs = gm[gm["CV"] == gm["CV"].max()].index.to_list()[0] + 1
    print("Best random_state number:", rs)
else:
    print("No suitable random state found.")


Best random_state number: 11


##  LogisticRegression

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=11)

In [34]:
from sklearn.linear_model import  LogisticRegression
log_model=  LogisticRegression()
log_model.fit(x_train,y_train)


ypred_train=log_model.predict(x_train)
ypred_test=log_model.predict(x_test)

print("Train Accuracy :",accuracy_score(y_train,ypred_train))
print("Cross Validation Score:",cross_val_score(log_model,x_train,y_train,
                                               cv=5,scoring='accuracy').mean())
print('Test Accuracy:',accuracy_score(y_test,ypred_test))

Train Accuracy : 0.9138305575669804
Cross Validation Score: 0.9037121331031235
Test Accuracy: 0.8815028901734104


## Ada Boost classifier

In [33]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

# Assuming rs, x_train, y_train, x_test, y_test are already defined somewhere in your code
estimator = AdaBoostClassifier(random_state=rs)
param_grid = {'n_estimators': list(range(1, 51))}  # Fixed typo: 'n_estimators'

# Perform GridSearchCV
ab_grid = GridSearchCV(estimator, param_grid, scoring='accuracy', cv=5)
ab_grid.fit(x_train, y_train)

# Get the best estimator and feature importances
ab = ab_grid.best_estimator_
ab_fi = ab.feature_importances_  # Fixed typo: 'feature_importances_'

# Select important features
index = [i for i, x in enumerate(ab_fi) if x > 0]

# Subset the training and test data with important features
x_train_ab = x_train.iloc[:, index]
x_test_ab = x_test.iloc[:, index]

# Fit the model with the important features
ab.fit(x_train_ab, y_train)  # Fixed typo: 'y_train'

# Make predictions
ypred_train = ab.predict(x_train_ab)
ypred_test = ab.predict(x_test_ab)

# Print accuracy scores
print("Train Accuracy:", accuracy_score(y_train, ypred_train))
print("Cross Validation Score:", cross_val_score(ab, x_train_ab, y_train, cv=5, scoring='accuracy').mean())
print("Test Accuracy:", accuracy_score(y_test, ypred_test))


Train Accuracy: 0.8479362780593772
Cross Validation Score: 0.8291110762308376
Test Accuracy: 0.8554913294797688
